In [1]:
import mlflow
import time
import torch

from omegaconf import OmegaConf
from torch.utils.data import DataLoader

from nba_ingame_prob.consts import proj_paths
from nba_ingame_prob.model.bayesian import BayesianResultPredictor
from nba_ingame_prob.model.inputs import prepare_scalers
from nba_ingame_prob.training.dataset import NBADataset
from nba_ingame_prob.training.pipeline import prepare_data
from nba_ingame_prob.training.run_torch import run_training

In [2]:
def process_key(key):
    return "".join(
        word[0]
        for word in key.split('_')
    )

def name_run(config):
    run_name = '_'.join(
        f"{value}{process_key(key)}"
        for key, value in config.training.model_config.items()
        if key != 'time_scaling'
    )
    run_name += f"_wd{config.training.params.weight_decay}"
    run_name += '_WL'  if config.training.params.weighted_loss else "_nWL"
    run_name += "_TS" if config.training.model_config.time_scaling else "_nTS"
    run_name += '_' + config.model_name.split('_')[0]
    return run_name

In [3]:
config

Available objects for config:
    AliasManager
    DisplayFormatter
    HistoryManager
    IPCompleter
    IPKernelApp
    LoggingMagics
    MagicsManager
    OSMagics
    PrefilterManager
    ScriptMagics
    StoreMagics
    ZMQInteractiveShell


In [4]:
now = time.strftime("%Y%m%d-%H%M%S")
config = OmegaConf.load(proj_paths.config.bayesian)
scalers_path = proj_paths.output / 'scalers.pkl'
input_teams = config['inputs_team'] or []
data_prepared = prepare_data(**config['data_params'])
scalers = prepare_scalers(data_prepared.X_train, input_teams, scalers_path)

train_dataset = NBADataset(input_teams, config['target'], data_prepared.X_train, data_prepared.teams, scalers)
test_dataset = NBADataset(input_teams, config['target'], data_prepared.X_test, data_prepared.teams, scalers)

params = config['training']['params']
train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_init = OmegaConf.create(dict(
    team_count=len(data_prepared.teams),
    team_features=len(input_teams)//2,
    **config['training']['model_config']
))
model_init_path = proj_paths.output / "model_init.yaml"
model_init_path.open("w").write(OmegaConf.to_yaml(model_init))
model = BayesianResultPredictor(**model_init).to(device)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=params['learning_rate'],
    weight_decay=params['weight_decay'] or 0.0
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = params.get('lr_gamma', 1.0))

create_experiment = mlflow.get_experiment_by_name(config['experiment_name']) is None
if create_experiment:
    mlflow.create_experiment(
        name=config['experiment_name'],
        tags={
            'author': 'Tom Bukic',
            'company': 'accelerate.ai'
        }
    )
    create_experiment = False

mlflow.set_experiment(config['experiment_name'])
with mlflow.start_run(run_name=f'{name_run(config)}-{now}'):
    mlflow.log_artifact(model_init_path)
    run_training(model, train_loader, test_loader, config, optimizer,  device, scheduler,  timestamp=now)


2025/09/13 00:27:42 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
loss: 1.3398 | epoch:  79/80: : 17200it [10:15, 27.94it/s]                         
2025/09/13 00:37:58 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/09/13 00:37:58 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run Noneed_4thd_1tl_4rhd_4rl_wd0.5_nWL_nTS_cb-20250913-002738 at: http://mlflow:5000/#/experiments/664344747479239756/runs/84e702a348d84f8b849ad8d7f38d80e1
🧪 View experiment at: http://mlflow:5000/#/experiments/664344747479239756
